# Preparation

In [ ]:
import os
import platform
import datetime
import numpy as np
import pandas as pd
import warnings
import re
import matplotlib.pyplot as plt
from scipy.io import loadmat
from pyts import *
import timeit

In [ ]:
if platform.system() == 'Windows':
    drive = 'D:'
elif platform.system() == 'Linux':
    drive = '/mnt/ntfs'
fileLoc = drive + ''

In [ ]:
neuronDF = pd.read_excel(drive + '',
                         sheet_name = None, usecols = ['SESSION', 'Neuron_Name'], engine = 'openpyxl')

In [ ]:
def formatData(data, time, timeLocks, binSize, maxLead, maxLag, method):
    
    if len(data) != len(time):
        raise Exception('Data and time must have equal length')

    binBounds = np.arange(-np.floor(maxLead/binSize),np.floor(maxLag/binSize)+1)*binSize
    
    nBins = len(binBounds) - 1
    nTrials = len(timeLocks)
    fData = np.zeros((nBins, nTrials))
    
    timeBounds = np.array(list(zip(binBounds[0:-1],binBounds[1:])))

    for iTrial in range(nTrials):

        for iBin in range(nBins):
            if method == 'count':
                fData[iBin, iTrial] = np.sum(data[(time >= (timeLocks[iTrial] + binBounds[iBin])) & (time < (timeLocks[iTrial] + binBounds[iBin + 1]))])
            elif method == 'firing_rate':
                isi = st[1:]-st[:-1]
                fData[iBin, iTrial] = np.mean(data[(time >= (timeLocks[iTrial] + binBounds[iBin])) & (time < (timeLocks[iTrial] + binBounds[iBin + 1]))])      
                
    return fData.T, timeBounds

In [ ]:
def cond2Label(condVec, condDict, condVals):
    cv = condVec.copy()
    indMask = np.array(np.zeros(len(condVec),), dtype = bool)
    
    for cond in condVals:
        indMask[condVec == cond] = True
    
    for cKey, cValues in condDict.items():
        for val in cValues:
            cv[cv == val] = cKey 
    
    return cv[indMask], indMask

In [ ]:
def prepEventDict(eventDict):
    
    eventTimes = []
    condVec = []
    
    for event in eventDict:
        
        if not isinstance(eventDict[event],np.ndarray):
            b = np.array(eventDict[event]).reshape(1,)
        else:
            b = eventDict[event]
        
        c = np.array([event]*len(b))
            
        eventTimes.append(b)
        condVec.append(c)
        
    eventTimes = np.concatenate(eventTimes, axis = 0)
    condVec = np.concatenate(condVec, axis = 0)

    return eventTimes, condVec

In [ ]:
def genDataDicts(fileLoc):
    
    folders = os.listdir(fileLoc)
    
    neuronDict = {}
    eventDict = {}
    prosEvents = {}
    prosEventDict = {}
    
    for folder in folders:
        
        folderLoc = fileLoc + '/' + folder
        files = os.listdir(folderLoc)

        if ('spk_lfp_evt_eog.mat' in files) & ('evt.mat' in files) & (len(folder) == 8):
        
            matSpikes = loadmat(folderLoc + '/' + 'spk_lfp_evt_eog.mat', simplify_cells = True)
            matEvents = loadmat(folderLoc + '/' + 'evt.mat', simplify_cells = True)

            for neuron in matSpikes['nexFile']['neurons']:
                nID = neuron['name']
                nID = nID.replace('Elec_', 'E')
                nID = nID.replace('_Neuron_', 'N')
                rDate = datetime.datetime.strptime(folder, '%d%m%Y').strftime('%d%m%y')
                neuronDict[rDate + nID] = {'timestamps': neuron['timestamps'], 'date': rDate, 'xPos': neuron['xPos'], 'yPos': neuron['yPos']}

            eventDict[rDate] = {}
            eventDict[rDate]['time'] = matEvents['evt']['time']
            eventDict[rDate]['cond'] = matEvents['evt']['cond']

            prosEvents[rDate] = {}
            for event in matSpikes['nexFile']['events']:
                prosEvents[rDate][event['name']] = event['timestamps']
                
            prosEventDict[rDate] = {}
            prosEventDict[rDate]['time'], prosEventDict[rDate]['cond'] = prepEventDict(prosEvents[rDate])
    
            
    return neuronDict, eventDict, prosEventDict

In [ ]:
def genDataDicts(fileLoc):
    
    folders = os.listdir(fileLoc)
    
    for folder in folders:
        
        folderLoc = fileLoc + '/' + folder
        files = os.listdir(folderLoc)

        if ('spk_lfp_evt_eog.mat' in files) & ('evt.mat' in files) & (len(folder) == 8):
        
            matEvents = loadmat(folderLoc + '/' + 'evt.mat', simplify_cells = True)
    
            
    return neuronDict, eventDict, prosEventDict

In [ ]:
def prepTrainingSet(neuronDict, eventDict, eventInd, pars, condDict, eventFile = True):
    
    selectedNeurons = []
    trialLengths = [0]
    condLengths = []
    condVals = []
    
    trainingSet = None
    eventConds = None

    for key, val in condDict.items():
        for cond in val:
            if cond not in condVals:
                condVals.append(cond)
    
    for neuronName, neuronData, in neuronDict.items():
        
        
        acDate = neuronData['date']
        timestamps = neuronData['timestamps']
        events = eventDict[acDate]['time']

        contCond = True
        
        for cv in condVals:
            if cv not in eventDict[acDate]['cond']:
                contCond = False
                
        if (type(timestamps) != float) and (len(timestamps) >= 30000) and contCond:
            
            e, condMask = cond2Label(eventDict[acDate]['cond'], condDict, condVals)
            
            if eventFile:
                tEvent = events[condMask][:,eventInd]
            else:
                tEvent = events[condMask]
                e = np.asarray(e, dtype = np.int64)

            n, _ = formatData(np.ones((len(timestamps))), timestamps, tEvent,
                    pars['binSize'], pars['maxLead'], pars['maxLag'], method = pars['method'])
            
            trialLengths.append(trialLengths[-1] + np.shape(e)[-1]) 
            cl = np.where(np.diff(e) != 0)[0]
            cl = list(trialLengths[-2] + cl)
            condLengths.append(trialLengths[-2])
            
            for temp in cl:
                condLengths.append(temp)
            selectedNeurons.append(neuronName)
            
        else:
            continue

        if trainingSet is None:
            trainingSet = n
            eventConds = e
        else:
            trainingSet = np.concatenate((trainingSet,n), axis = 0)
            eventConds = np.concatenate((eventConds,e), axis = 0)
            
    
    condLengths.append(trialLengths[-1])

    return trainingSet, eventConds, trialLengths, condLengths, selectedNeurons


## Testing

In [ ]:
cDict = {0: ['Cnd_1_Feedback_1'], 1: ['Cnd_3_Feedback_1']}
pars = {'binSize':0.1, 'maxLead':3, 'maxLag':3, 'method':'count'}
eInd = 6

In [ ]:
cDict = {0:[1], 1:[3]}
pars = {'binSize':0.1, 'maxLead':4, 'maxLag':2, 'method':'count'}
eInd = 6

nDict, eDict, pEDict = genDataDicts(fileLoc)
tSet2, tLab2, tLens2, condLens2, sNeur2 = prepTrainingSet(nDict, eDict, eInd, pars, cDict)


neuronInds2 = np.array(list(zip(tLens2[:-1],tLens2[1:])))

testSet2 = tSet2

for trialInd in neuronInds2:
    tt = tSet2[trialInd[0]:trialInd[1]]
    # transformer = skl.preprocessing.QuantileTransformer(output_distribution = 'normal').fit(tt)
    transformer = skl.preprocessing.StandardScaler().fit(tt)
    testSet2[trialInd[0]:trialInd[1]] = transformer.transform(tt)

testSet2 = np.sum(testSet2, axis = 1)
testSet2 = testSet2.reshape(-1,1)

condInds2 = np.array(list(zip(condLens2[:-1],condLens2[1:])))

nSet2 = np.zeros((len(condInds2),))

for cInd, cVal in enumerate(condInds2):
    tt = testSet2[cVal[0]:cVal[1]]
    nSet2[cInd] = np.mean(tt)
tt = np.array(list(zip(nSet2[::2],nSet2[1::2])))